In [1]:
# created on Dec 24, 2020
# modified on April 12, 2022
# @author:          Bo Zhao
# @email:           zhaobo@uw.edu
# @website:         https://hgis.uw.edu
# @organization:    Department of Geography, University of Washington, Seattle
# @description:     Search geo-tagged tweets within the U.S. This script is modified from https://github.com/shawn-terryah/Twitter_Geolocation

In [2]:
import tweepy, json, time
import pandas as pd
from google.colab import files
# Create data on to Google Drive
from google.colab import drive
# Mount your Drive to the Colab VM.
drive.mount('/gdrive')

Mounted at /gdrive


In [3]:
class StreamListener(tweepy.StreamListener):
    """tweepy.StreamListener is a class provided by tweepy used to access
    the Twitter Streaming API to collect tweets in real-time.
    """

    def __init__(self, time_limit=60, file=""):
        """class initialization"""
        self.start_time = time.time()
        self.limit = time_limit
        self.result = []
        self.f = file
        super(StreamListener, self).__init__()

    def on_data(self, data):
        """This is called when data are streamed in."""
        if (time.time() - self.start_time) < self.limit:
            datajson = json.loads(data)
            # print(datajson, "\n")
            if 'id' not in datajson.keys():
                time.sleep(10)
            else:
                id = datajson['id']
                username = datajson['user']['screen_name']
                created_at = datajson['created_at']
                text = datajson['text'].strip().replace("\n", "")

                # process the geo-tags
                if datajson['coordinates'] == None:
                    try:
                        bbox = datajson['place']['bounding_box']['coordinates'][0]
                        lng = (bbox[0][0] + bbox[2][0]) / 2.0
                        lat = (bbox[0][1] + bbox[1][1]) / 2.0
                    except:
                        lat = 0
                        lng = 0
                else:
                    lng = datajson['coordinates']['coordinates'][0]
                    lat = datajson['coordinates']['coordinates'][1]

                if lat != 0:
                    row = {
                        'id': id,
                        'username': username,
                        'created_at': created_at,
                        'lng': lng,
                        'lat': lat,
                        'text': text
                    }
                    print(row)
                    self.result.append(row)
                else:
                    pass
        else:
            df = pd.DataFrame(self.result)
            df.to_csv(self.f, index=False)
            # download the csv to your local computer
            files.download(self.f)
            print("the csv has been downloaded to your local computer. The program has been completed successfully.")
            return False

In [4]:
if __name__ == "__main__":
    # These are provided to you through the Twitter API after you create a account
    # register a Twitter App to get the keys and access tokens.
    output_file = '/gdrive/My Drive/geotweets.csv'

    # Apply for your own Twitter API keys at https://developer.twitter.com/en/apply-for-access
    consumer_key = "consumer_key"
    consumer_secret = "consumer_secret"
    access_token = "access_token"
    access_token_secret = "access_token_secret"

    myauth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    myauth.set_access_token(access_token, access_token_secret)

    # LOCATIONS are the longitude, latitude coordinate corners for a box that restricts the
    # geographic area from which you will stream tweets. The first two define the southwest
    # corner of the box and the second two define the northeast corner of the box.
    LOCATIONS = [-124.7771694, 24.520833, -66.947028, 49.384472,  # Contiguous US (location #1)
                 -164.639405, 58.806859, -144.152365, 71.76871,  # Alaska (^)
                 -160.161542, 18.776344, -154.641396, 22.878623]  # Hawaii (^)
                 #-12.068606, 49.744176, 2.555234, 58.764420] # United Kingdom and Ireland (location #2)


    stream_listener = StreamListener(time_limit=600, file=output_file)
    stream = tweepy.Stream(auth=myauth, listener=stream_listener)
    stream.filter(track=['celtics'])
    stream.filter(locations=LOCATIONS, languages=['en'], encoding="utf-8")

{'id': 1533659197899829249, 'username': 'hooperstarium', 'created_at': 'Mon Jun 06 03:57:01 +0000 2022', 'lng': -115.223125, 'lat': 36.232915, 'text': 'Looney tunes "pitch perfect" as Poole &amp; the @warriors silence the @celtics 107-88 in #NBAFinals Game 2. https://t.co/M5FHdSjb6H'}
{'id': 1533659209920675841, 'username': 'ThisKidB', 'created_at': 'Mon Jun 06 03:57:04 +0000 2022', 'lng': -122.2286855, 'lat': 37.791994, 'text': 'Hahahaha you stuck in purgatory.. he do make the offense look disjointed though'}
{'id': 1533659232033067009, 'username': 'sashadigiulian', 'created_at': 'Mon Jun 06 03:57:09 +0000 2022', 'lng': -105.5508905, 'lat': 38.997935999999996, 'text': 'Celtics maybe should have eaten a PERFORM bar. But there’s always RECOVER now… https://t.co/BN1WRavCDj 🍋 🍒 🥜 #NBAFinals'}
{'id': 1533659772431413248, 'username': 'CEOofYEEZUS', 'created_at': 'Mon Jun 06 03:59:18 +0000 2022', 'lng': -122.529191, 'lat': 45.432278499999995, 'text': 'I need this tee🔥🔥'}
{'id': 1533659800118

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

the csv has been downloaded to your local computer. The program has been completed successfully.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

the csv has been downloaded to your local computer. The program has been completed successfully.
